In [42]:
#Code copied 100% (besides the structure name) from
#https://stackoverflow.com/questions/49356018/how-to-calculate-the-average-structure-of-a-protein-with-multiple-models-conform

#In order to use, you must make a model PDB structure that contains each structure you want to average one after another
#Currently this is done manually

import Bio.PDB
import numpy as np

pdbid='NAME'

parser = Bio.PDB.PDBParser(QUIET=True)  # Don't show me warnings
structure = parser.get_structure(pdbid, pdbid+'.pdb')  # id of pdb file and location

atoms = [a.parent.parent.id + '-' + str(a.parent.id[1]) + '-' +  a.name for a in structure[0].get_atoms() if a.parent.id[0] == ' ']  # obtained from model '0'

atom_avgs = {}
for atom in atoms:
    atom_avgs[atom] = []
    for model in structure:
        atom_ = atom.split('-')
        coor = model[atom_[0]][int(atom_[1])][atom_[2]].coord
        atom_avgs[atom].append(coor)
    atom_avgs[atom] = sum(atom_avgs[atom]) / len(atom_avgs[atom])  # average
    
ns = Bio.PDB.StructureBuilder.Structure('id='+pdbid)  #  new structure
ns.add(structure[0])  # add model 0
for atom in ns[0].get_atoms():
    chain = atom.parent.parent
    res = atom.parent
    if res.id[0] != ' ':
        chain.detach_child(res)  # detach hetres
    else:
        coor = atom_avgs[chain.id + '-' + str(res.id[1]) + '-' + atom.name]
        atom.coord = coor
        
io = Bio.PDB.PDBIO()
io.set_structure(ns)
io.save('avg_'+pdbid+'.pdb')